In [9]:
import pickle
from tensorflow.keras.models import load_model

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
working_dir = os.getenv("WORKING_DIR")
os.chdir(working_dir)

In [11]:
model = load_model('saved_models/model.h5')

with open('saved_models/le_gender.pkl' , 'rb') as file:
    le = pickle.load(file)

with open('saved_models/ohe_geography.pkl' , 'rb') as file:
    ohe = pickle.load(file)

with open('saved_models/scaler.pkl' , 'rb') as file:
    scaler = pickle.load(file)

In [12]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame([input_data])

In [15]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [16]:
df['Gender'] = le.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
ohe_geo = ohe.transform(df[['Geography']])

In [18]:
df_ohe = pd.DataFrame(ohe_geo.toarray() , columns=ohe.get_feature_names_out())

In [19]:
df_ohe

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
df = pd.concat([df.drop('Geography' , axis = 1) , df_ohe] , axis = 1)

In [21]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [22]:
df = scaler.transform(df)

In [23]:
prediction = model.predict(df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step


array([[0.02680411]], dtype=float32)

In [24]:
prediction_proba = prediction[0][0]
if prediction_proba > .5:
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

Customer is not likely to churn
